In [45]:
#import libraries
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.optimizers import Adam

import random
np.random.seed(32)
random.seed(32)

In [46]:
# Load the MNIST Fashion dataset
filename = 'fashion-mnist.csv'
df = pd.read_csv(filename)

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# Normalize pixel values between [0, 1]
X = X / 255

# Only use 50% of data
rand = random.sample(range(10000), 5000)
X = X[rand]
y = y[rand]

# One-hot encoding
y = to_categorical(y)

# Split the dataset into training and test data sets
X_left, X_gen, y_left, y_gen = train_test_split(X, y, test_size=0.2, random_state=420)
X_train, X_test, y_train, y_test = train_test_split(X_left, y_left, test_size=0.25, random_state=420)

In [47]:
# Define the CNN model
def create_model(learning_rate, dropout_rate, num_filters, num_layers):
    model = Sequential()
    model.add(Conv2D(num_filters, (3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    for i in range(num_layers):
        model.add(Conv2D(num_filters, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    optimizer = Adam(learning_rate = learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [48]:
# Define the search space for the hyperparameters
search_space = [Real(1e-4, 1e-2, name='learning_rate'),
                Real(0.1, 0.5, name='dropout_rate'),
                Integer(16, 64, name='num_filters'),
                Integer(1, 3, name='num_layers')]

In [49]:
# Define the objective function to minimize (i.e., 1 - accuracy)
acc_list = []

@use_named_args(search_space)
def objective(**params):
    # Create the model
    model = create_model(**params)

    # Fit the model to the training data
    model.fit(X_train.reshape(-1, 28, 28, 1), y_train, epochs=10, verbose=0)

    # Predict the labels for the test data
    y_pred = model.predict(X_test.reshape(-1, 28, 28, 1))

    # Convert the predicted labels to integers
    y_pred = np.argmax(y_pred, axis=1)

    # Compute the accuracy of the model
    accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)

    #print(accuracy)
    acc_list.append(accuracy)

    # Return 1 - accuracy (to minimize)
    return 1.0 - accuracy

In [50]:
# Perform Bayesian optimization
global prob_list_bo
prob_list_bo = []
result = gp_minimize(objective, search_space, n_calls=10, random_state=420, acq_func="gp_hedge")

# Print the best set of hyperparameters and the corresponding accuracy
print(f'Best hyperparameters: {result.x}')
print(f'Best accuracy: {1.0 - result.fun}')
print(prob_list_bo)

32/32 [==============================] - 0s 5ms/step


ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_126. Consider increasing the input size. Received input shape [None, 1, 1, 33] which would produce output shape with a zero or negative value in a dimension.